Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
import pickle
import sys

C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-

In [2]:
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
# Function to show various error measurement
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    mse = np.mean((forecast - actual)**2)  # MSE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    #corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #mins = np.amin(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #maxs = np.amax(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]                      # ACF1
    #return({'mape':mape, 'me':me, 'mae': mae, 
    #        'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
    #        'corr':corr, 'minmax':minmax})
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'mse':mse, 'rmse':rmse})

In [4]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


In [5]:
file_dir = './Data Exploration/data/uc3/'
X_train_file_path = 'all_hosp_new_train_X_uc3.pkl'
y_train_file_path = 'all_hosp_data_uc3_y_train_uc3.pkl'
X_test_file_path = 'all_hosp_new_test_X_uc3.pkl'
y_test_file_path = 'all_hosp_data_uc3_y_test_uc3.pkl'

In [6]:
df_X_train = pd.read_pickle(file_dir + X_train_file_path)
df_y_train = pd.read_pickle(file_dir + y_train_file_path)
df_X_test = pd.read_pickle(file_dir + X_test_file_path)
df_y_test = pd.read_pickle(file_dir + y_test_file_path)
print(df_X_train.shape, df_y_train.shape, df_X_test.shape, df_y_test.shape)

(46888, 14313) (46888,) (15629, 14313) (15629,)


In [7]:
print(df_y_test.values.shape)

(15629,)


Neural Networks ( Sequential )

In [8]:
NN_model = Sequential()
n_hid_nodes = 4096
dropout_rate = 0.2
rand_seed = 42
kernel_init = 'he_normal'
hid_act = 'relu'

In [9]:
NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init,input_dim = df_X_train.shape[1], activation=hid_act))
NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [10]:
#NN_model.add(GaussianNoise(0.01,input_shape = (df_X_train.shape[1],)))
#NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init,activation='relu'))
#NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [11]:
for i in range(5):
    NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init))#,activation=hid_act))
    NN_model.add(BatchNormalization())
    NN_model.add(Activation(hid_act))
    NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [12]:
NN_model.add(Dense(1, kernel_initializer=kernel_init,activation='linear'))

In [13]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              58630144  
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
activation (Activation)      (None, 4096)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              1

In [14]:
checkpoint_name = 'NN2_Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [15]:
NN_model.fit(df_X_train, df_y_train, epochs=100, batch_size=128, validation_split = 0.2, callbacks=callbacks_list)

# save model to file
NN_model.save(file_dir + "uc3_NN2_model.h5")

Train on 37510 samples, validate on 9378 samples
Epoch 1/100
37504/37510 [============================>.] - ETA: 0s - loss: 6259.8815 - mean_absolute_error: 6259.8794
Epoch 00001: val_loss improved from inf to 6408.00158, saving model to NN2_Weights-001--6408.00158.hdf5
37510/37510 [==============================] - 78s 2ms/sample - loss: 6259.4697 - mean_absolute_error: 6259.4673 - val_loss: 6408.0016 - val_mean_absolute_error: 6408.0020
Epoch 2/100
37504/37510 [============================>.] - ETA: 0s - loss: 4413.5970 - mean_absolute_error: 4413.5977
Epoch 00002: val_loss improved from 6408.00158 to 2955.14515, saving model to NN2_Weights-002--2955.14515.hdf5
37510/37510 [==============================] - 75s 2ms/sample - loss: 4414.3641 - mean_absolute_error: 4414.3647 - val_loss: 2955.1452 - val_mean_absolute_error: 2955.1455
Epoch 3/100
37504/37510 [============================>.] - ETA: 0s - loss: 2679.6331 - mean_absolute_error: 2679.6326
Epoch 00003: val_loss improved from 29

37504/37510 [============================>.] - ETA: 0s - loss: 1041.7340 - mean_absolute_error: 1041.7335
Epoch 00023: val_loss improved from 1812.72225 to 1810.02216, saving model to NN2_Weights-023--1810.02216.hdf5
37510/37510 [==============================] - 76s 2ms/sample - loss: 1041.6844 - mean_absolute_error: 1041.6841 - val_loss: 1810.0222 - val_mean_absolute_error: 1810.0223
Epoch 24/100
37504/37510 [============================>.] - ETA: 0s - loss: 1032.8754 - mean_absolute_error: 1032.8754
Epoch 00024: val_loss did not improve from 1810.02216
37510/37510 [==============================] - 71s 2ms/sample - loss: 1032.9775 - mean_absolute_error: 1032.9774 - val_loss: 1853.3946 - val_mean_absolute_error: 1853.3945
Epoch 25/100
37504/37510 [============================>.] - ETA: 0s - loss: 997.0445 - mean_absolute_error: 997.0445
Epoch 00025: val_loss improved from 1810.02216 to 1805.76201, saving model to NN2_Weights-025--1805.76201.hdf5
37510/37510 [=========================

Epoch 47/100
37504/37510 [============================>.] - ETA: 0s - loss: 697.2471 - mean_absolute_error: 697.2472
Epoch 00047: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 76s 2ms/sample - loss: 697.8335 - mean_absolute_error: 697.8336 - val_loss: 1885.1812 - val_mean_absolute_error: 1885.1813
Epoch 48/100
37504/37510 [============================>.] - ETA: 0s - loss: 689.1867 - mean_absolute_error: 689.1867
Epoch 00048: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 86s 2ms/sample - loss: 689.7184 - mean_absolute_error: 689.7184 - val_loss: 1817.9817 - val_mean_absolute_error: 1817.9817
Epoch 49/100
37504/37510 [============================>.] - ETA: 0s - loss: 694.7743 - mean_absolute_error: 694.7743
Epoch 00049: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 71s 2ms/sample - loss: 695.5213 - mean_absolute_error: 695.5212 - val_loss: 1829.0633 - val_mean_abs

37504/37510 [============================>.] - ETA: 0s - loss: 551.5898 - mean_absolute_error: 551.5898
Epoch 00071: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 70s 2ms/sample - loss: 552.0482 - mean_absolute_error: 552.0482 - val_loss: 1842.6370 - val_mean_absolute_error: 1842.6372
Epoch 72/100
37504/37510 [============================>.] - ETA: 0s - loss: 563.7759 - mean_absolute_error: 563.7761
Epoch 00072: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 70s 2ms/sample - loss: 564.5251 - mean_absolute_error: 564.5254 - val_loss: 1846.1016 - val_mean_absolute_error: 1846.1016
Epoch 73/100
37504/37510 [============================>.] - ETA: 0s - loss: 551.5939 - mean_absolute_error: 551.5941
Epoch 00073: val_loss did not improve from 1801.08387
37510/37510 [==============================] - 70s 2ms/sample - loss: 552.5615 - mean_absolute_error: 552.5616 - val_loss: 1860.5096 - val_mean_absolute_error: 

MemoryError: Unable to allocate 14.0 MiB for an array with shape (128, 14313) and data type float64

In [ ]:
predictions = NN_model.predict(df_X_test)

In [ ]:
sns.regplot(x=df_y_test, y=predictions)

In [ ]:
# MSE
print('Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=True))

# RMSE
print('Root Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=False))

# MAE
print('Mean absolute error: %.2f'
      % mean_absolute_error(df_y_test,predictions))

In [ ]:
forecast_accuracy(predictions.reshape((-1,)), df_y_test)